- text emotion detection with machine learning
- my dataset is more like a transcript not written by someone (it is not a chat, it is a verbal conversation)
- emo2vec
- NRC Emotion intensity lexicon
- emotional word/context/sentnce emdedding for sentiment analysis
- sentence embedding for emotional recognition
- sentiment embedding (?)
- emotion recognition in time

In [1]:
import os

import csv
import json

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score,matthews_corrcoef

from tqdm import tqdm, trange,tnrange,tqdm_notebook
import random
import os
import io
% matplotlib inline


ModuleNotFoundError: No module named 'torch'

In [ ]:
def openESConv():
    """Function to open the ESConv.json file to pd.DataFrame with tables: ConversationID and Seeker Dialog"""
    
    file_path = "ESConv.json"

    with open(file_path, "r", encoding="utf-8") as file:
        dataset = json.load(file)
        
    corpus_data = []
    emotion_type_list = []
    conv_id = 0

    for corpus in dataset:
        dialog = corpus['dialog']
        corpus_data.append(dialog)
        conv_id += 1
        emotion_type = corpus['emotion_type']
        emotion_type_list.append([conv_id, emotion_type])


    df = pd.DataFrame({'Dialog': pd.Series(corpus_data)})

    seeker_df = pd.DataFrame(emotion_type_list, columns=['ConversationID', 'Seeker Dialog', 'Emotion Type'])

    for index, row in df.iterrows():
        dialog_list = row['Dialog']
        for dialog in dialog_list:
            if dialog['speaker'] == 'seeker':
                seeker_df = seeker_df.append({'ConversationID': index, 'Seeker Dialog': dialog['content']}, ignore_index=True)
    return seeker_df
df = openESConv()

In [ ]:
!pip install pytorch

In [ ]:
df.head()

## Spliting the dataset into two sets:

1. Training Set: Used to train the model (70-80% of original dataset)

2. Testing Set: Used to get an unbiased estimate of the model performance (20-30% of original dataset)

In [ ]:
train, test = train_test_split(df, test_size=0.2, random_state=0)

In [ ]:
test.head()

In [ ]:
print(train.shape, test.shape)

In [ ]:
!pip install transformers

In [ ]:
# identify and specify the GPU as the device, later in training loop we will load data into device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

SEED = 19

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == torch.device("cuda"):
    torch.cuda.manual_seed_all(SEED)

## "Machine Learning and Natural Language Processing in Mental Health" Le Glaz et al.

Methodology:

1.) Preprocessing - "analyzing the data to obtain numeric or categorical features"

2.) Classification

## 1.) Preprocessing:
- lemma
- POS (part of speach)
- cTAKES (clinical Text Analysis and Knowledge Extraction System)
- ngrams
- tfidf
- embedding
- CUIs (concept unique identifier)
- LDA (latent Dirichlet allocation)
- SentiAna (analyzing sentiments or emotions)
- LIWC (Linguistic Inquiry and Word Count)
- NER (named entity recognition)
- LSA (latent semantic analysis)

## 2.) Classification
- SVM
- Logistic Regression
- Random Forest
- Linear Regression
- Neural Network